# 题目

> 实现一个 MyCalendar 类来存放你的日程安排。如果要添加的日程安排不会造成重复预订，则可以存储这个新的日程安排。  
当两个日程安排有一些时间上的交叉时（例如两个日程安排都在同一时间内），就会产生重复预订。  
日程可以用一对整数 start 和 end 表示，这里的时间是半开区间，即 [start, end)，实数 x 的范围为， start <= x < end 。  
实现 MyCalendar 类：  
MyCalendar() 初始化日历对象。  
boolean book(int start, int end) 如果可以将日程安排成功添加到日历中而不会导致重复预订，返回 true 。否则，返回 false 并且不要将该日程安排添加到日历中。

# 方法一：线段树

> 线段树：  
https://blog.csdn.net/zearot/article/details/48299459

> 利用线段树，假设开辟了数组 $arr[0,⋯ ,10^9]$ ，初始时每个元素的值都为 0 ，对于每次行程预订的区间 $[start,end)$ ，则我们将区间中的元素 $arr[start,⋯ ,end−1]$ 中的每个元素都标记为 1 ，每次调用 book 时，我们只需要检测 $arr[start,⋯ ,end−1]$ 区间内是否有元素被标记为 1 。实际我们不必实际开辟数组 $arr$ ，可采用动态线段树，懒标记 $lazy$ 标记区间 $[l,r]$ 已经被预订， tree 记录区间 $[l,r]$ 的是否存在标记为 1 的元素。  

> 每次进行 book 操作时，首先判断区间 $[start,⋯ ,end−1]$ 是否存在元素被标记，如果存在被标记为 1 的元素，则表明该区间不可预订；否则，则将可以预订。预订完成后，将 $arr[start,⋯ ,end−1]$ 进行标记为 1 ，并同时更新线段树。

> 线段树可以解决大量区间问题。

## 复杂度

- 时间复杂度: $O(nlogC)$ ，其中 $n$ 是日程数量， $C$ 是线段树区间的长度。

> 由于使用了线段树查询，线段树的最大深度为 $log⁡C$ ，每次最多会查询 $log⁡C$ 个节点，每次求最大的预订需的时间复杂度为 $O(log⁡C+log⁡C)$ ，因此时间复杂度为 $O(nlog⁡C)$ ，在此 $C$ 取固定值 $10^9$ 。

- 空间复杂度: $O(nlogC)$ ，其中 $n$ 是日程数量， $C$ 是线段树区间的长度。

> 由于该解法采用的为动态线段树，线段树的最大深度为 $log⁡C$ ，每次预订最多会在线段树上增加 $log⁡C$ 个节点，因此空间复杂度为 $O(nlog⁡C)$ ，在此 $C$ 取固定值 $10^9$ 。

## 代码

In [1]:
class MyCalendar:
    def __init__(self):
        self.tree = set()
        self.lazy = set()

    # 线段树操作：查询某个区间是否在线段树中，返回布尔值
    # l为线段树左边界，r为线段树右边界，idx表示某段日期的编号
    # [0,1e9)编号idx=1，[0,mid)编号idx=2, [mid,1e9)编号idx=3，以此类推
    # 所以idx实际相当于线段树里的节点编号
    def query(self, start, end, l, r, idx):
        # 若当前日程超出线段树边界，直接返回False
        if r < start or end < l:
            return False
        # 如果该区间已被预订，则返回True
        if idx in self.lazy:
            return True
        # 如果当前的[l,r]是当前日程的子区间（找到一个刚好被当前日程覆盖的时间段）
        if start <= l and r <= end:
            # 返回：idx是否在tree中
            return idx in self.tree
        # 计算当前的中点
        mid = (l + r) // 2
        # 分别在左右子树里继续查询
        return self.query(start, end, l, mid, 2 * idx) or self.query(start, end, mid + 1, r, 2 * idx + 1)

    # 线段树操作：输入一段区间，更新tree和lazy
    def update(self, start, end, l, r, idx):
        # 若当前日程超出线段树边界，直接返回None
        if r < start or end < l:
            return
        # 如果当前的[l,r]是当前日程的子区间（找到一个刚好被当前日程覆盖的时间段）
        if start <= l and r <= end:
            # 记录存在已预订的日程的区间的idx
            self.tree.add(idx)
            # 对其进行预定，记录目标无法被其他日程预定的区间的idx
            self.lazy.add(idx)
        # 若没找到这样的时间段，则在左右子树继续查找
        else:
            # 递归地查找左右子树
            mid = (l + r) // 2
            self.update(start, end, l, mid, 2 * idx)
            self.update(start, end, mid + 1, r, 2 * idx + 1)
            # 递归过程中，记录所有包含该日程的区间的idx
            self.tree.add(idx)
            # 递归过程中，若发现当前区间的左右子区间都被预定，则预定当前区间
            if 2 * idx in self.lazy and 2 * idx + 1 in self.lazy:
                self.lazy.add(idx)

    # 解法函数
    def book(self, start, end):
        # 查询区间是否已被预定
        if self.query(start, end - 1, 0, 10 ** 9, 1):
            # 若是，说明该区间无法预定
            return False
        # 若区间没被预定，更新
        self.update(start, end - 1, 0, 10 ** 9, 1)
        return True

#### 测试一

In [2]:
myCalendar = MyCalendar()
print(myCalendar.book(10, 20))
print(myCalendar.book(15, 25))
print(myCalendar.book(20, 30))

True
False
True
